In [1]:
### --- Импорт библиотек ---

# === Стандартная библиотека ===
import sys
import math
from io import StringIO, BytesIO
from datetime import datetime
from dateutil.relativedelta import relativedelta

# === Аналитика и обработка данных ===
import numpy as np
import pandas as pd

# === Визуализация ===
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image

# === Прогресс-бары ===
from tqdm import tqdm

# === Работа с файлами ===
# from openpyxl import load_workbook
from pyxlsb import open_workbook

# === Работа с базами данных ===
from clickhouse_driver import dbapi
from simplech import ClickHouse
# import psycopg2  # если используешь PostgreSQL

# === Работа с API и интеграции ===
import telebot

# === Google API ===
import gspread as gs
from gspread.client import ExportFormat
from google.oauth2 import service_account
from googleapiclient.discovery import build

# === Пользовательские функции ===
# from functions_dwh import (
#     ch_query_DWH2,
#     ch_query_mp,
#     pg_query_mostbetcom,
#     write_data
# )

In [2]:
# Ячейка если необходимо доустановить какие-то библиотеки
# pip install gspread

In [3]:
# подставить - Нужную дату !!!!

start_date = pd.to_datetime('2025-06-19') # Год месяц день

In [4]:
# Устанавливаем опцию для отображения до 100 строк
pd.set_option('display.max_rows', 100)

# Устанавливаем опцию для отображения до 100 колонок
pd.set_option('display.max_columns', 100)

In [5]:
### Настраиваем соединение с БД
creds = pd.read_excel('Data_base.xlsx', index_col=0)

db_connection = ClickHouse(host='data.base.example', port='8123',
                     user=creds.loc['data.base.example', 'user'],
                     password=creds.loc['data.base.example', 'password'])

def ch_query_DWH2(query, index=None):
    query += ' FORMAT CSVWithNames'
    result = db_connection.select(query)
    df = pd.read_csv(StringIO(result), low_memory=False)
    if index:
        df.set_index(index, inplace=True)
    return df

# Start:

In [6]:
# список партнеров продукта
black_list_payback_DWH_data = ch_query_DWH2(f"""
    SELECT *
    FROM reports.black_list_payback
""")

In [7]:
black_list_payback_DWH_data.shape[0]

3034

# Перестраховка перед манипуляциями в скрипте, сохраняем первоначальный валинор!!

In [8]:
black_list_payback_DWH_data.to_excel('blp_ПЕРВОНАЧАЛЬНЫЙ_ПЕРЕСТРАХОВКА.xlsx', index=False)

# Конец и продолжение

In [9]:
fresh_webs_to_block_list = pd.read_excel('Блок_2.xlsx', dtype={'sub': 'str'})

In [10]:
# Замена пропусков в определенных колонках
columns_to_replace = ['sub', 'sub_type']
black_list_payback_DWH_data[columns_to_replace] = black_list_payback_DWH_data[columns_to_replace].fillna("")
fresh_webs_to_block_list[columns_to_replace] = fresh_webs_to_block_list[columns_to_replace].fillna("")

In [11]:
# Указание новых названий колонок
columns = list(black_list_payback_DWH_data.columns)

# Создание DataFrame с указанными колонками
# Установка новых названий колонок
fresh_webs_to_block_list.columns = columns

In [12]:
#### указываем актуальный тип данных для БД

fresh_webs_to_block_list = fresh_webs_to_block_list.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

black_list_payback_DWH_data = black_list_payback_DWH_data.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

In [13]:
### base 
black_list_payback_DWH_data.head(1)

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,perform,AZ,49660,ferid_abdullayev,sub3,4,1980-01-01,Скам!


In [14]:
black_list_payback_DWH_data.shape[0]

3034

In [15]:
### new
fresh_webs_to_block_list.head(1)

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,perform,RU,347114,a627,sub2,1,2025-06-18,Медиабаинг: Трафик остановлен


# 1 Объединим два датасета в один:

In [16]:
# Объединяем DataFrame по строкам
concat_data_payback = pd.concat([black_list_payback_DWH_data, 
                                 fresh_webs_to_block_list], 
                                ignore_index=True)

# просто проверка
assert concat_data_payback.isna().sum().sum() == 0
assert (
        black_list_payback_DWH_data.shape[0] 
    + 
        fresh_webs_to_block_list.shape[0] 
    == 
        concat_data_payback.shape[0]
       )

In [17]:
for_check = concat_data_payback.shape[0]
concat_data_payback.shape[0]

3047

In [18]:
true_columns_list = list(concat_data_payback.columns)

In [19]:
concat_data_payback = concat_data_payback.drop(columns=['sub_type'])

In [20]:
# Убираем временно всех, кто не прошел тесты
concat_data_payback = (
                        concat_data_payback[concat_data_payback['block_desc'] 
                    != 
                        'Не успешная тестовая закупка']
                    .reset_index(drop=True)
                      )

# Подгружаем источники не прошедшие тесты

In [21]:
# список партнеров продукта
stopped_test_partners = ch_query_DWH2(f"""
SELECT 
        added_dt AS blocked_date, 
        partner_id AS pid, 
        in_model_sub_id AS sub
    FROM sandbox_analytical_projects.stopped_test_partners
    WHERE recommendation == 'Прекратить тест'
""")

In [22]:
stopped_test_partners['sub'] = stopped_test_partners['sub'].fillna('')

In [23]:
assert stopped_test_partners.isna().sum().sum() == 0

In [24]:
stopped_test_partners['block_flg'] = 1

In [25]:
stopped_test_partners['block_desc'] = 'Не успешная тестовая закупка'

In [26]:
stopped_test_partners.head()

,blocked_date,pid,sub,block_flg,block_desc
0,2025-06-13 17:09:37,49706,arabasevdalilari.tr,1,Не успешная тестовая закупка
1,2025-06-13 17:09:37,49706,brawlstarseditsturkiye,1,Не успешная тестовая закупка
2,2025-06-13 17:09:37,49706,corba.tarifciniz,1,Не успешная тестовая закупка
3,2025-06-13 17:09:37,49706,kartal.reyiz,1,Не успешная тестовая закупка
4,2025-06-13 17:09:37,364890,konserdozu,1,Не успешная тестовая закупка


In [27]:
assert stopped_test_partners.shape[0] != 0

In [28]:
concat_data_payback.tail(1)

,project,type,geo,pid,sub,block_flg,blocked_date,block_desc
1382,mb,perform,KZ,126898,richads - push,2,2025-06-19,limit: 1600; речек 02.07.2025


# Берем Тип саба из справочника

In [29]:
# список партнеров продукта
mediabuying_info____ = ch_query_DWH2(f"""
SELECT 
        site, 
        partner_id AS pid, 
        sub_type, 
        `type`, 
        geo
    FROM reports.mediabuying_info 
    WHERE business_unit = 'Acq'
""")

In [30]:
mediabuying_info____['sub_type'] = mediabuying_info____['sub_type'].fillna('')

In [32]:
assert mediabuying_info____.isna().sum().sum() == 0

In [33]:
mediabuying_info____ = (
                            mediabuying_info____.
                    loc[(mediabuying_info____['site'] == 'Beta') 
                        | 
                        (mediabuying_info____['site'] == 'Mos')]
                    .reset_index(drop=True)
                       )

In [34]:
def assign_value(row):
    if row['site'] in ['mb', 'Mos']:
        return 'mb'
    elif row['site'] == 'Beta':
        return 'ba'
    else:
        return 'na'

# Применение функции к DataFrame для создания новой колонки
mediabuying_info____['project'] = mediabuying_info____.apply(assign_value, axis=1)

In [35]:
assert mediabuying_info____[mediabuying_info____['project'] == 'na'].shape[0] == 0

In [36]:
# xxx = stopped_test_partners.shape[0]
stopped_test_partners = stopped_test_partners.merge(mediabuying_info____, on=['pid'], how='left') 
################################ Удаление строк, где в колонке1 пропуски
stopped_test_partners = stopped_test_partners.dropna(subset=['geo']).reset_index(drop=True)
###################################################
# assert xxx == stopped_test_partners.shape[0]
assert stopped_test_partners.isna().sum().sum() == 0

In [37]:
stopped_test_partners.loc[(stopped_test_partners['sub'] == ''), 'sub_type'] = ''

In [38]:
stopped_test_partners = stopped_test_partners.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

In [39]:
stopped_test_partners.loc[(stopped_test_partners['geo'] == 'Multigeo'), 'geo'] = 'ALL'

In [41]:
mediabuying_info____ = mediabuying_info____[['pid', 'sub_type', 'project']]

In [42]:
mediabuying_info____.head()

,pid,sub_type,project
0,2314,,ba
1,2318,sub1,ba
2,337502,,mb
3,6686,sub2,ba
4,6690,sub2,ba


Блок дополнительной проверки расчетов:

In [43]:

xxx = concat_data_payback.shape[0]
concat_data_payback = concat_data_payback.merge(mediabuying_info____, on=['pid', 'project'], how='left') 
assert xxx == concat_data_payback.shape[0]
concat_data_payback.loc[(concat_data_payback['sub'] == ''), 'sub_type'] = ''
assert concat_data_payback.isna().sum().sum() == 0

In [44]:
xxx == concat_data_payback.shape[0]

True

In [45]:
concat_data_payback[concat_data_payback['sub_type'].isna()]

,project,type,geo,pid,sub,block_flg,blocked_date,block_desc,sub_type


In [46]:
concat_data_payback.isna().sum()

project         0
type            0
geo             0
pid             0
sub             0
block_flg       0
blocked_date    0
block_desc      0
sub_type        0
dtype: int64

In [47]:
concat_data_payback.isna().sum()

project         0
type            0
geo             0
pid             0
sub             0
block_flg       0
blocked_date    0
block_desc      0
sub_type        0
dtype: int64

In [48]:
concat_data_payback

,project,type,geo,pid,sub,block_flg,blocked_date,block_desc,sub_type
0,mb,perform,AZ,49660,ferid_abdullayev,4,1980-01-01,Скам!,sub3
1,mb,brand,AZ,135157,QAQASBUNETEMADI,4,1980-01-01,Скам!,sub1
2,mb,perform,KZ,64146,rakhmetoovvaa,4,1980-01-01,Скам!,sub2
3,mb,perform,KZ,64146,isamakarov,4,1980-01-01,Скам!,sub2
4,mb,perform,KZ,64146,niko_nett,4,1980-01-01,Скам!,sub2
...,...,...,...,...,...,...,...,...,...
1378,mb,perform,KG,264474,kadam,2,2025-06-19,limit: 1800; речек 02.07.2025,sub2
1379,mb,perform,RU,114168,2230,2,2025-06-19,limit: 2000; речек 02.07.2025,sub2
1380,mb,brand,ALL,351438,,1,2025-06-19,Период мониторинга окончен. Трафик не окупаем,
1381,mb,perform,TR,49706,anlik.goalss,1,2025-06-19,Медиабаинг: Трафик остановлен,sub1


In [49]:
concat_data_payback.loc[(concat_data_payback['sub'] == ''), 'sub_type'] = ''

In [50]:
concat_data_payback = concat_data_payback[true_columns_list]
stopped_test_partners = stopped_test_partners[true_columns_list]

In [51]:
##### Объединям с источниками не прошедшими тест!!!!

In [52]:
xxx = concat_data_payback.shape[0]
# Объединяем DataFrame по строкам
concat_data_payback = pd.concat([concat_data_payback, stopped_test_partners], ignore_index=True)

# просто проверка
assert concat_data_payback.isna().sum().sum() == 0
assert xxx + stopped_test_partners.shape[0] == concat_data_payback.shape[0]

In [53]:
concat_data_payback.head(1)

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,perform,AZ,49660,ferid_abdullayev,sub3,4,1980-01-01,Скам!


In [54]:
concat_data_payback.shape[0]

4674

In [55]:
for_check

3047

In [56]:
# Проверка того что мы все ок сделали

assert concat_data_payback.shape[0] >= for_check

# Убираем временно весть 4й флаг:

In [57]:
# Убираем временно весть Антифрод
concat_data_payback = concat_data_payback[concat_data_payback['block_flg'] != 4].reset_index(drop=True)

In [58]:
concat_data_payback.head()

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,perform,RU,114170,1740,sub2,0,2024-05-08,Исторические данные по остановленным источникам
1,mb,perform,RU,114170,1717,sub2,0,2024-05-08,Исторические данные по остановленным источникам
2,mb,perform,PK,126906,trafficstars,sub2,0,2024-05-08,Исторические данные по остановленным источникам
3,mb,perform,KZ,116858,,,0,2024-05-08,Исторические данные по остановленным источникам
4,mb,perform,MA,156541,igetmedia,sub2,0,2024-05-08,Исторические данные по остановленным источникам


# Разделяем на датасет с флагом 0 и остальные флаги

In [59]:
concat_data_payback_ZERO_FLG = (
                                concat_data_payback[concat_data_payback['block_flg'] == 0]
                                .reset_index(drop=True)
                                    )

concat_data_payback_ANOTHER_FLG = (
                                    concat_data_payback[concat_data_payback['block_flg'] != 0]
                                    .reset_index(drop=True)
                                      )

In [61]:
### Дубликаты если по не нулевым флагам, 
### которые необходимо дополнительно согласовывать с заказчиками !

concat_data_payback_ANOTHER_FLG[concat_data_payback_ANOTHER_FLG.duplicated(
    subset=['project', 
            'type', 
            'geo', 
            'pid', 
            'sub', 
            'sub_type', 
            'block_flg'], 
    keep=False
)].sort_values(by=['pid', 'sub'])

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
2132,mb,brand,UZ,25716,Historical_NaN,sub3,1,2024-09-27 18:21:18,Не успешная тестовая закупка
2612,mb,brand,UZ,25716,Historical_NaN,sub3,1,2024-12-06 18:27:35,Не успешная тестовая закупка
1185,ba,perform,ALL,25744,,,1,2025-01-14 18:39:17,Не успешная тестовая закупка
1186,ba,perform,ALL,25744,,,1,2025-01-14 18:39:17,Не успешная тестовая закупка
776,mb,brand,BR,49448,Historical_NaN,sub2,1,2024-11-08 18:52:28,Не успешная тестовая закупка
...,...,...,...,...,...,...,...,...,...
1079,mb,brand,BD,356716,Historical_NaN,sub2,1,2024-12-13 17:23:19,Не успешная тестовая закупка
1230,mb,perform,ALL,360176,,,1,2025-01-21 17:23:05,Не успешная тестовая закупка
1231,mb,perform,ALL,360176,,,1,2025-01-21 17:23:05,Не успешная тестовая закупка
3337,mb,brand,ALL,363778,,,1,2025-04-04 17:32:03,Не успешная тестовая закупка


In [62]:
#### полноe удаление дублей !

In [63]:
concat_data_payback_ANOTHER_FLG.shape[0]

3501

In [64]:
### удаление дубликатов
concat_data_payback_ANOTHER_FLG = (
                                    concat_data_payback_ANOTHER_FLG
                                .drop_duplicates()
                                .reset_index(drop=True)
                                      )

In [65]:
# Общая сверка строк таблицы после чистки дублей
concat_data_payback_ANOTHER_FLG.shape[0]

2308

# РАЗОБЬЕМ НЕ НУЛЕВЫЕ ФЛАГИ НА 1 И ОСТАЛЬНЫЕ

In [66]:
concat_data_payback_1_FLG = (
                                concat_data_payback_ANOTHER_FLG[
                                    concat_data_payback_ANOTHER_FLG['block_flg'] == 1
                                        ].reset_index(drop=True)
                                )
concat_data_payback_NOT_ONE_FLG = (
                                    concat_data_payback_ANOTHER_FLG[
                                        concat_data_payback_ANOTHER_FLG['block_flg'] != 1
                                            ].reset_index(drop=True)
                                          )

# очистим строки, в которых уже есть 1 (удалим их по новой дате, оставив только старую)

In [67]:
#concat_data_payback_1_FLG

concat_data_payback_1_FLG[concat_data_payback_1_FLG.duplicated(
    subset=['project', 
            'type', 
            'geo', 
            'pid', 
            'sub', 
            'sub_type', 
            'block_flg'], 
    keep=False
)].sort_values(by=['pid', 'sub', 'geo'])

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
1415,mb,brand,UZ,25716,Historical_NaN,sub3,1,2024-09-27 18:21:18,Не успешная тестовая закупка
1486,mb,brand,UZ,25716,Historical_NaN,sub3,1,2024-12-06 18:27:35,Не успешная тестовая закупка
432,mb,brand,BR,49448,Historical_NaN,sub2,1,2024-11-08 18:52:28,Не успешная тестовая закупка
462,mb,brand,BR,49448,Historical_NaN,sub2,1,2024-11-15 17:03:42,Не успешная тестовая закупка
477,mb,brand,BR,49448,Historical_NaN,sub2,1,2024-11-22 18:07:25,Не успешная тестовая закупка
...,...,...,...,...,...,...,...,...,...
542,mb,perform,BR,350732,Historical_NaN,sub2,1,2024-12-13 17:23:19,Не успешная тестовая закупка
1488,mb,perform,BR,350732,Historical_NaN,sub2,1,2024-12-06 18:27:35,Не успешная тестовая закупка
486,mb,perform,TR,351244,Historical_NaN,sub2,1,2024-11-22 18:07:25,Не успешная тестовая закупка
502,mb,perform,TR,351244,Historical_NaN,sub2,1,2024-11-29 17:33:21,Не успешная тестовая закупка


In [68]:
concat_data_payback_1_FLG = concat_data_payback_1_FLG.sort_values(
    by=['project', 
        'type', 
        'geo', 
        'pid', 
        'sub', 
        'sub_type', 
        'block_flg', 
        'blocked_date'], 
    ascending=[True, 
               True, 
               True, 
               True, 
               True, 
               True, 
               True, 
               True]  # Дата по убыванию
).reset_index(drop=True)
.drop_duplicates(
    subset=['project', 
            'type', 
            'geo', 
            'pid', 
            'sub', 
            'sub_type', 
            'block_flg'], 
    keep='first'  # Оставляем первую (самую старшую) дату
).reset_index(drop=True)

In [70]:
### Ячейка для ручной проверки конкретного партнера, не более
concat_data_payback_1_FLG.loc[(concat_data_payback_1_FLG['pid']==49624) 
                              & (concat_data_payback_1_FLG['sub']=='ig10')]

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
709,mb,brand,UZ,49624,ig10,sub3,1,2025-04-11 15:43:38,Не успешная тестовая закупка


In [71]:
### с флагами 1 все ок в этом сегмете идем дальше!

# теперь занулим лишние не нулевые флаги, которые не равны 1

In [72]:
#concat_data_payback_NOT_ONE_FLG


#### Визуальный осмотр дублей по флагам, удовлетворяющим условия если такие есть !
concat_data_payback_NOT_ONE_FLG[concat_data_payback_NOT_ONE_FLG.duplicated(
    subset=['project', 
            'type', 
            'geo', 
            'pid', 
            'sub', 
            'sub_type', 
            'block_flg'], 
    keep=False
)].sort_values(by=['pid', 'sub', 'geo'])

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc


In [73]:
# Найдем дубликаты по заданным столбцам
duplicates = concat_data_payback_NOT_ONE_FLG.duplicated(
    subset=['project', 
            'type', 
            'geo', 
            'pid', 
            'sub', 
            'sub_type', 
            'block_flg'], 
    keep=False  
)

# Сортируем строки с дубликатами по blocked_date (по возрастанию)
concat_data_payback_NOT_ONE_FLG.loc[duplicates] = concat_data_payback_NOT_ONE_FLG.loc[duplicates] \
    .sort_values(by=['blocked_date'], ascending=True)

# Обнуляем block_flg для всех строк-дубликатов, кроме самой поздней
concat_data_payback_NOT_ONE_FLG.loc[
    duplicates & concat_data_payback_NOT_ONE_FLG.duplicated(
        subset=['project', 
                'type', 
                'geo', 
                'pid', 
                'sub', 
                'sub_type', 
                'block_flg'], 
        keep='last'  # Оставляем последнюю
    ),
    'block_flg'
] = 0

# Если есть флаг 1, то остальные флаги должны быть равны нулю

In [75]:
# Объединение по ключевым столбцам
merged_df = concat_data_payback_NOT_ONE_FLG.merge(concat_data_payback_1_FLG,
                                                  on=['project', 
                                                      'type', 
                                                      'geo', 
                                                      'pid', 
                                                      'sub', 
                                                      'sub_type'], 
                                                  how='left', 
                                                  suffixes=('', '_df1'))

In [77]:
#### Кейсы где дужно занулить 2й и 3й флаг, т.к. есть первый(Если такие есть)!!!!
FLAG_2_3_BY_ZERO = merged_df.loc[(merged_df['block_flg_df1'] == 1)]

In [78]:
#### Кейсы где дужно занулить 2й и 3й флаг, т.к. есть первый(Если такие есть)!!!!
merged_df.loc[(merged_df['block_flg_df1'] == 1)]

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc,block_flg_df1,blocked_date_df1,block_desc_df1


In [79]:
# Создаем словарь из merged_df (где block_flg == 1)
mask = merged_df['block_flg_df1'] == 1
pid_to_update = merged_df.loc[mask, ['project', 
                                     'type', 
                                     'geo', 
                                     'pid', 
                                     'sub', 
                                     'sub_type']]
                          .set_index(
                                        ['project', 
                                         'type', 
                                         'geo', 
                                         'pid', 
                                         'sub', 
                                         'sub_type']
).index

# Обновляем block_flg
concat_data_payback_NOT_ONE_FLG.loc[
    concat_data_payback_NOT_ONE_FLG.set_index(['project', 
                                               'type', 
                                               'geo', 
                                               'pid', 
                                               'sub', 
                                               'sub_type']).index.isin(pid_to_update),
    'block_flg'
] = 0


# объединим все назад в один датасет, что бы не запутаться

In [81]:
# Объединяем DataFrame по строкам
concat_data_payback = pd.concat([concat_data_payback_1_FLG, 
                                 concat_data_payback_NOT_ONE_FLG], 
                                ignore_index=True).reset_index(drop=True)
concat_data_payback = pd.concat([concat_data_payback, 
                                 concat_data_payback_ZERO_FLG], 
                                ignore_index=True).reset_index(drop=True)

In [82]:
assert concat_data_payback.isna().sum().sum() == 0

In [84]:
concat_data_payback[concat_data_payback['block_flg'] == 2].head()

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
1880,mb,perform,RU,347114,988,sub2,2,2025-04-23,limit=200
1881,mb,brand,UZ,321992,charos__gafurova,sub3,2,2025-05-07,limit=1100; речек 04.06.2025
1882,mb,perform,RU,114170,st346,sub2,2,2025-05-21,limit=3800; речек 18.06.2025
1883,mb,brand,BD,340702,1913,sub2,2,2025-05-21,limit=1200; речек 18.06.2025
1884,mb,perform,PT,295150,,,2,2025-05-21,limit=7500; речек 04.06.2025


# Занулим флаг 2 для дублей, если они есть во флаге 3

In [85]:
concat_data_payback_2_FLG = (
                                concat_data_payback[
                                    concat_data_payback['block_flg'] == 2]
                                .reset_index(drop=True)
                                    )
concat_data_payback_3_FLG = (
                                concat_data_payback[
                                    concat_data_payback['block_flg'] == 3]
                                .reset_index(drop=True)
                                    )


concat_data_payback_1_0_FLG = (concat_data_payback.loc[
                                                        (concat_data_payback['block_flg'] != 2) 
                                                      & (concat_data_payback['block_flg'] != 3)
                                                    ].reset_index(drop=True)
                                      )

In [86]:
# Объединение по ключевым столбцам
merged_df = concat_data_payback_2_FLG.merge(concat_data_payback_3_FLG, on=['project', 
                                                                           'type', 
                                                                           'geo', 
                                                                           'pid', 
                                                                           'sub', 
                                                                           'sub_type'], 
                                            how='left', 
                                            suffixes=('', '_df1')
                                                   )


In [88]:
# По аналогии
mask = merged_df['block_flg_df1'] == 3
pid_to_update = merged_df.loc[mask, ['project', 
                                     'type', 
                                     'geo', 
                                     'pid', 
                                     'sub', 
                                     'sub_type']]
                         .set_index(
                                    ['project', 
                                     'type', 
                                     'geo', 
                                     'pid', 
                                     'sub', 
                                     'sub_type']
).index

# Обновляем block_flg в concat_data_payback_2_FLG
concat_data_payback_2_FLG.loc[
    concat_data_payback_2_FLG.set_index(['project', 
                                         'type', 
                                         'geo', 
                                         'pid', 
                                         'sub', 
                                         'sub_type'])
    .index.isin(pid_to_update),
    'block_flg'
] = 0


In [90]:
# Объединяем DataFrame по строкам
concat_data_payback = pd.concat([concat_data_payback_2_FLG, 
                                 concat_data_payback_3_FLG], 
                                ignore_index=True).reset_index(drop=True)
# Объединяем DataFrame по строкам
concat_data_payback = pd.concat([concat_data_payback, 
                                 concat_data_payback_1_0_FLG], 
                                ignore_index=True).reset_index(drop=True)

In [91]:
assert (concat_data_payback.shape[0] == (concat_data_payback_2_FLG.shape[0] 
                                      +  concat_data_payback_3_FLG.shape[0] 
                                      + concat_data_payback_1_0_FLG.shape[0]
                                            )
               )

assert concat_data_payback.isna().sum().sum() == 0

In [94]:
# Визуальный осмотр
concat_data_payback[concat_data_payback['block_flg'] == 2].head()

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,perform,RU,347114,988,sub2,2,2025-04-23,limit=200
1,mb,brand,UZ,321992,charos__gafurova,sub3,2,2025-05-07,limit=1100; речек 04.06.2025
2,mb,perform,RU,114170,st346,sub2,2,2025-05-21,limit=3800; речек 18.06.2025
3,mb,brand,BD,340702,1913,sub2,2,2025-05-21,limit=1200; речек 18.06.2025
4,mb,perform,PT,295150,,,2,2025-05-21,limit=7500; речек 04.06.2025


# Добавление новых записей из таблицы Антифрода по заблокированным АФ источникам:

In [97]:
true_columns_list = list(concat_data_payback.columns)

In [98]:
strange_partners_try = pg_query_mostbetcom(f"""
                                            SELECT 
                                                    partner_id, 
                                                    updated_dwh as updated_at, 
                                                    traffic_source, 
                                                    geo, 
                                                    case 
                                                        when sub in ('NA', 'ALL') 
                                                        then '' else sub end sub
                                                FROM web_analytics.antifraud.strange_partners
                                                WHERE decision = 'Блокировка АФ' --is_fraud_af=1
                                                ORDER BY updated_at desc
                                                    """).fillna('')

/home/jovyan/work/!!!!!Новые_Красные_вебы!!!!!!/!!!ПАПКА_С_ИТОГОВЫМ_ВАРИАНТОМ_РАБОТЫ/!!!ДЛЯ_АНТИФРОДА_ЗАБЛОКИРОВАННЫЕ_ВЕБЫ/functions_dwh.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=engine)


In [99]:
assert strange_partners_try.shape[0] != 0

In [100]:
strange_partners_try = (strange_partners_try[strange_partners_try['updated_at'].notna()]
                        .reset_index(drop=True)
                               )

In [101]:
strange_partners_try.head(1)

,partner_id,updated_at,traffic_source,geo,sub
0,49534,2025-06-18 18:29:09.952824,,IN,23044


In [102]:
strange_partners_try = strange_partners_try[['geo', 
                                             'partner_id', 
                                             'sub', 
                                             'updated_at']].copy()

In [103]:
strange_partners_try['block_desc'] = 'Антифрод: Заблокировано'

In [104]:
strange_partners_try = strange_partners_try.astype({
    'geo': 'str',
    'partner_id': 'int64',
    'sub': 'str',
    'updated_at': 'datetime64[ns]',
    'block_desc': 'str'
})

assert strange_partners_try.isna().sum().sum() == 0

In [105]:
### подгрузка инфо
# список партнеров продукта
mediabuying_info = ch_query_DWH2(f"""
    SELECT *
        FROM reports.mediabuying_info
        WHERE business_unit ILIKE '%acqu%'
""").fillna('')

In [106]:
mediabuying_info = mediabuying_info[['site', 'partner_id', 'geo', 'sub_type', 'type']].copy()

In [107]:
# Функция для присвоения значений новой колонке
def assign_value(row):
    if row['site'] == 'Mos':
        return 'mb'
    elif row['site'] == 'Beta':
        return 'ba'
    elif row['site'] == 'Bnai':
        return 'bnai'    
    else:
        return 'vi'  

# Применение функции к DataFrame для создания новой колонки
mediabuying_info['project'] = mediabuying_info.apply(assign_value, axis=1)
__mediabuying_info__ = mediabuying_info.copy()
__mediabuying_info__['project'] = __mediabuying_info__.apply(assign_value, axis=1)

In [108]:
mediabuying_info = mediabuying_info[['project', 
                                     'type', 
                                     'geo', 
                                     'partner_id', 
                                     'sub_type']].copy()

In [109]:
mediabuying_info_MONO_GEO = mediabuying_info[mediabuying_info['geo'] != 'Multigeo'].reset_index(drop=True)

In [110]:
strange_partners_TRUE = mediabuying_info_MONO_GEO.merge(strange_partners_try[['geo', 
                                                                              'partner_id', 
                                                                              'sub', 
                                                                              'updated_at', 
                                                                              'block_desc']], 
                                                        on=['partner_id', 
                                                            'geo'], 
                                                        how='left')

In [111]:
## фильтр значений не являющихся пустыми
strange_partners_TRUE = strange_partners_TRUE[strange_partners_TRUE['sub'].notna()]

In [112]:
## проверка дублей в таблице АФ
strange_partners_TRUE[strange_partners_TRUE.duplicated()]

,project,type,geo,partner_id,sub_type,sub,updated_at,block_desc
138,mb,perform,RU,114170,sub2,1901,2025-02-28 19:58:03.888546,Антифрод: Заблокировано


In [113]:
## детальная проверка дублей (Необходимо выяснить отдельно причину их появления)
strange_partners_TRUE.loc[(strange_partners_TRUE['sub'].isin(['BA', '9136']))]

,project,type,geo,partner_id,sub_type,sub,updated_at,block_desc
186,ba,perform,AZ,139509,sub3,BA,2025-02-28 19:58:03.888546,Антифрод: Заблокировано
187,ba,perform,AZ,139513,sub3,BA,2025-02-28 19:58:03.888546,Антифрод: Заблокировано


In [114]:
strange_partners_TRUE.shape[0]

187

In [115]:
### удаление дубликатов
strange_partners_TRUE = strange_partners_TRUE.drop_duplicates().reset_index(drop=True)



In [116]:
strange_partners_TRUE.shape[0]

186

In [117]:
#### Теперь нужно добавить мультигео пидов

In [118]:
mediabuying_info_Multigeo = mediabuying_info[mediabuying_info['geo'] == 'Multigeo'].reset_index(drop=True)

In [119]:
strange_partners_Multigeo = (strange_partners_try.loc[strange_partners_try['partner_id']
                              .isin(list(mediabuying_info_Multigeo['partner_id'].unique()))]
                              .reset_index(drop=True)
                                    )

In [120]:
strange_partners_Multigeo

,geo,partner_id,sub,updated_at,block_desc
0,RU,229549,1782,2025-03-21 14:39:00.212406,Антифрод: Заблокировано
1,BD,295150,BRpwaMBD013,2025-02-28 19:58:03.888546,Антифрод: Заблокировано


In [121]:
strange_partners_Multigeo.shape[0]

2

In [122]:
strange_partners_Multigeo = strange_partners_Multigeo.merge(mediabuying_info_Multigeo[['project', 
                                                                                       'type', 
                                                                                       'partner_id', 
                                                                                       'sub_type']], 
                                                            on=['partner_id'], 
                                                            how='left')

In [123]:
strange_partners_Multigeo.shape[0]

2

In [124]:
strange_partners_Multigeo = strange_partners_Multigeo.rename(columns={'updated_at': 'blocked_date',
                                                                       'partner_id': 'pid'})

strange_partners_TRUE = strange_partners_TRUE.rename(columns={'updated_at': 'blocked_date',
                                                               'partner_id': 'pid'})

In [125]:
strange_partners_Multigeo['block_flg'] = 1
strange_partners_TRUE['block_flg'] = 1

In [126]:
strange_partners_Multigeo = strange_partners_Multigeo[true_columns_list].copy()

strange_partners_TRUE = strange_partners_TRUE[true_columns_list].copy()

In [127]:
strange_partners_Multigeo = strange_partners_Multigeo.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

assert strange_partners_Multigeo.isna().sum().sum() == 0


strange_partners_TRUE = strange_partners_TRUE.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

assert strange_partners_TRUE.isna().sum().sum() == 0

In [128]:
_strange_partners_TRUE_ = pd.concat([strange_partners_TRUE, 
                                     strange_partners_Multigeo], 
                                    ignore_index=True)

In [129]:
assert _strange_partners_TRUE_.shape[0] == (
                                            strange_partners_TRUE.shape[0] 
                                          + strange_partners_Multigeo.shape[0]
                                                   )

In [130]:

_strange_partners_TRUE_ = _strange_partners_TRUE_.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

assert _strange_partners_TRUE_.isna().sum().sum() == 0

In [131]:
###### Удалим возможные дубли по АФ
_strange_partners_TRUE_ = _strange_partners_TRUE_.drop_duplicates().reset_index(drop=True)

In [132]:
_strange_partners_TRUE_['block_flg'] = 4

In [133]:
_strange_partners_TRUE_['block_desc'].unique()

array(['Антифрод: Заблокировано'], dtype=object)

In [134]:
_strange_partners_TRUE_.head(1)

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,brand,IN,49534,23044,sub2,4,2025-06-18 18:29:09.952824,Антифрод: Заблокировано


In [135]:
##### По антифроду датасет готов!!!!!!!!!

## Добавляем Скам пидов из гугл файла!!!!!!

In [136]:
# получение учетных данных 
# и изменение их на ваш адрес электронной почты 
# для использования delegated_credentials в дальнейшем коде
creds = service_account.Credentials.from_service_account_file(
        'client_secret.json', # тут client_secret.json от сервисного
        scopes=['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
    )

# masking service account with work account
delegated_creds = creds.with_subject('d.user@mail.cloud')


##########################################################################################



def google_sheet_df(doc_url: str, worksheet: str):
    gc = gs.authorize(delegated_creds)
    g_sheet = gc.open_by_url(doc_url)
    
    # Получаем все записи с 7-й строки как заголовками
    df = pd.DataFrame(g_sheet.worksheet(worksheet).get_all_records(head=1))  
    # Указано, что заголовки находятся на 7-й строке

    return df

doc_url = 'https://docs.google.com/spreadsheets/d/LLEzDgvqvDWCOzgSqKzgvqvDWCZYA'
worksheet = 'Лист1'



In [137]:
pids_list_data = google_sheet_df(doc_url, worksheet)

In [138]:
assert pids_list_data.shape[0] != 0

In [139]:
pids_list_data

,гео,пид,саб,причина/комментарий,менеджер
0,AZ,49660,ferid_abdullayev,"взял деньги, рекламу не сделал, пропал",Amin Bagirov
1,AZ,135157,QAQASBUNETEMADI,"взял деньги, рекламу не сделал, пропал",Nigar Aliyeva
2,KZ,64146,rakhmetoovvaa,после оплаты перестала выходить на связь,Алина Шмакова
3,KZ,64146,isamakarov,"после оплаты перестала выходить на связь, пере...",Алина Шмакова
4,KZ,64146,niko_nett,"не выполняет обещания, манипулирует ценой, ден...",Алина Шмакова
5,KZ,64146,kuanyweva_,"взял деньги, рекламу не сделал, пропал",Алина Шмакова
6,TR,49706,ig174,сделал одну публикацию после оплаты и перестал...,Melis Öz
7,TR,49706,ig175,сделал одну публикацию после оплаты и перестал...,Melis Öz
8,IN,49534,23044,"нет кликов и фд, но много регистраций",ALI SHAH MUHAMMAD
9,TR,49710,bonuslion,"плохо делает размещения, делает меньше обещанн...",Umut


In [140]:
pids_list_data = pids_list_data.rename(columns={'гео': 'geo',
                                               'пид': 'pid',
                                               'саб': 'sub'})

In [141]:
pids_list_data = pids_list_data[[
    'geo', 'pid', 'sub'
]].copy()

In [142]:
pids_list_data['blocked_date'] = '1980-01-01'
pids_list_data['block_desc'] = 'Скам!'
pids_list_data['block_flg'] = 4

In [143]:
pids_list_data = pids_list_data.astype({
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

assert pids_list_data.isna().sum().sum() == 0

In [144]:
pids_list_data

,geo,pid,sub,blocked_date,block_desc,block_flg
0,AZ,49660,ferid_abdullayev,1980-01-01,Скам!,4
1,AZ,135157,QAQASBUNETEMADI,1980-01-01,Скам!,4
2,KZ,64146,rakhmetoovvaa,1980-01-01,Скам!,4
3,KZ,64146,isamakarov,1980-01-01,Скам!,4
4,KZ,64146,niko_nett,1980-01-01,Скам!,4
5,KZ,64146,kuanyweva_,1980-01-01,Скам!,4
6,TR,49706,ig174,1980-01-01,Скам!,4
7,TR,49706,ig175,1980-01-01,Скам!,4
8,IN,49534,23044,1980-01-01,Скам!,4
9,TR,49710,bonuslion,1980-01-01,Скам!,4


In [145]:
#### Добавляем колонку project и sub_type из инфо !!!!

In [146]:
__mediabuying_info__ = __mediabuying_info__.rename(columns={'partner_id': 'pid'})

In [148]:
pids_list_data = pids_list_data.merge(
    __mediabuying_info__[['project', 'type', 'pid', 'sub_type']], on=['pid'], how='left')

In [149]:
pids_list_data = pids_list_data[_strange_partners_TRUE_.columns].copy()

In [150]:
pids_list_data = pids_list_data.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

assert pids_list_data.isna().sum().sum() == 0

In [151]:
pids_list_data.head(1)

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,perform,AZ,49660,ferid_abdullayev,sub3,4,1980-01-01,Скам!


In [152]:
##### Закончили со скамом!!!!!!!!!!

## Продолжаем дальнейшие манипуляции

In [154]:
# Объединение по ключевым столбцам
merged_df = _strange_partners_TRUE_.merge(pids_list_data, on=['project', 
                                                              'type', 
                                                              'geo', 
                                                              'pid', 
                                                              'sub', 
                                                              'sub_type'], 
                                          how='left', 
                                          suffixes=('', '_df1'))


In [155]:
merged_df

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc,block_flg_df1,blocked_date_df1,block_desc_df1
0,mb,brand,IN,49534,23044,sub2,4,2025-06-18 18:29:09.952824,Антифрод: Заблокировано,4.0,1980-01-01,Скам!
1,mb,perform,IN,49688,3015,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано,NaN,NaT,NaN
2,mb,perform,IN,49688,3012,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано,NaN,NaT,NaN
3,mb,perform,IN,49688,3013,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано,NaN,NaT,NaN
4,mb,perform,IN,49688,3082,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
183,mb,perform,KG,250176,kechkikabar24,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано,NaN,NaT,NaN
184,mb,brand,BD,356714,ProfessorOfPcGaming,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано,NaN,NaT,NaN
185,mb,brand,BD,356714,DGProduction,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано,NaN,NaT,NaN
186,ba,perform,RU,229549,1782,sub2,4,2025-03-21 14:39:00.212406,Антифрод: Заблокировано,NaN,NaT,NaN


In [156]:
#### Кейсы где дужно занулить АФ флаг и т.к. есть дубль в скаме!!!!
merged_df.loc[(merged_df['block_desc_df1'] == 'Скам!')]

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc,block_flg_df1,blocked_date_df1,block_desc_df1
0,mb,brand,IN,49534,23044,sub2,4,2025-06-18 18:29:09.952824,Антифрод: Заблокировано,4.0,1980-01-01,Скам!


In [157]:
# Создаем словарь из merged_df (где block_flg == Скам!)
mask = merged_df['block_desc_df1'] == 'Скам!'
pid_to_update = merged_df.loc[mask, ['project', 
                                     'type', 
                                     'geo', 
                                     'pid', 
                                     'sub', 
                                     'sub_type']]
                         .set_index(
                                    ['project', 
                                     'type', 
                                     'geo', 
                                     'pid', 
                                     'sub', 
                                     'sub_type']
).index

# Обновляем block_flg
_strange_partners_TRUE_.loc[
    _strange_partners_TRUE_.set_index(['project', 
                                       'type', 
                                       'geo', 
                                       'pid', 
                                       'sub', 
                                       'sub_type']).index.isin(pid_to_update),
    'block_flg'
] = 0


In [158]:
# Просто для доп проверки ячейка по возможным AD-Hoc!
_strange_partners_TRUE_.loc[(_strange_partners_TRUE_['pid'] == 49660) 
                            & (_strange_partners_TRUE_['sub'] == 'ferid_abdullayev')]

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc


In [159]:
_strange_partners_TRUE_.shape[0]

188

In [160]:
_strange_partners_TRUE_ = _strange_partners_TRUE_[_strange_partners_TRUE_['block_flg'] != 0]

In [161]:
_strange_partners_TRUE_.shape[0]

187

# Конкатим итоговый 4й флаг!!!!

In [164]:
# Объединяем DataFrame по строкам
base_data_to_update = pd.concat([_strange_partners_TRUE_, 
                                    pids_list_data], 
                                   ignore_index=True).reset_index(drop=True)

# Объединяем DataFrame по строкам
base_data_to_update = pd.concat([base_data_to_update, 
                                    concat_data_payback], 
                                   ignore_index=True).reset_index(drop=True)

In [165]:
assert (_strange_partners_TRUE_.shape[0] 
        + pids_list_data.shape[0] 
        + concat_data_payback.shape[0]) == base_data_to_update.shape[0]

In [166]:
base_data_to_update['block_flg'].unique()

array([4, 2, 3, 1, 0])

In [167]:
base_data_to_update.head()

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,perform,IN,49688,3015,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано
1,mb,perform,IN,49688,3012,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано
2,mb,perform,IN,49688,3013,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано
3,mb,perform,IN,49688,3082,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано
4,mb,perform,IN,49688,3081,sub2,4,2025-02-28 19:58:03.888546,Антифрод: Заблокировано


In [168]:
assert base_data_to_update.isna().sum().sum() == 0

# Зануляем все не нулевые флаги, если есть 4й флаг!!!!!!

In [169]:
##### Делим на два датасета с 4м флагом и всеми остальными!!!!!!!!!!

base_data_to_update_4_DATA = (base_data_to_update[base_data_to_update['block_flg'] == 4]
                                     .reset_index(drop=True)
                                        )

base_data_to_update_ANOTHER_FLG_DATA = (
                                        base_data_to_update[base_data_to_update['block_flg'] != 4]
                                           .reset_index(drop=True)
                                                  )

In [170]:
# Объединение по ключевым столбцам
merged_df = base_data_to_update_4_DATA.merge(base_data_to_update_ANOTHER_FLG_DATA, 
                                                on=['project', 
                                                    'type', 
                                                    'geo', 
                                                    'pid', 
                                                    'sub', 
                                                    'sub_type'], 
                                                how='left', 
                                                suffixes=('', '_df1'))


In [173]:
# Создаем словарь из merged_df (где block_flg == 4)
mask = (merged_df['block_flg_df1'] != 4) & (merged_df['block_desc_df1'].notna())
pid_to_update = merged_df.loc[mask, ['project', 
                                     'type', 
                                     'geo', 
                                     'pid', 
                                     'sub', 
                                     'sub_type']]
                                                .set_index(
                                                    ['project', 
                                                     'type', 
                                                     'geo', 
                                                     'pid', 
                                                     'sub', 
                                                     'sub_type']
).index

# Обновляем block_flg в concat_data_payback_2_FLG
base_data_to_update_ANOTHER_FLG_DATA.loc[
    base_data_to_update_ANOTHER_FLG_DATA.set_index(['project', 
                                                       'type', 
                                                       'geo', 
                                                       'pid', 
                                                       'sub', 
                                                       'sub_type']).index.isin(pid_to_update),
    'block_flg'
] = 0


In [174]:
# Для возможных AD-Hoc !
base_data_to_update_ANOTHER_FLG_DATA.loc[(base_data_to_update_ANOTHER_FLG_DATA['pid'] == 114170)
                                            & (base_data_to_update_ANOTHER_FLG_DATA['sub'] == 'st114')]

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
1452,mb,perform,RU,114170,st114,sub2,0,2025-03-19 17:21:15,Не успешная тестовая закупка


In [175]:
# Объединяем DataFrame по строкам
base_data_to_update = pd.concat([base_data_to_update_ANOTHER_FLG_DATA, 
                                    base_data_to_update_4_DATA], 
                                   ignore_index=True).reset_index(drop=True)

In [176]:
assert base_data_to_update.shape[0] == (
                                            base_data_to_update_ANOTHER_FLG_DATA.shape[0]
                                          + base_data_to_update_4_DATA.shape[0]
                                                  )

In [ ]:
# Визуальная проверка ниже :

In [177]:
base_data_to_update.head(1)

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,perform,RU,347114,988,sub2,2,2025-04-23,limit=200


In [178]:
base_data_to_update['block_flg'].unique()

array([2, 3, 1, 0, 4])

In [179]:
base_data_to_update.shape[0]

3095

# Продолжаем!

In [181]:
base_data_to_update.shape[0]

3095

In [182]:
base_data_to_update.head(1)

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,perform,RU,347114,988,sub2,2,2025-04-23,limit=200


In [183]:
assert base_data_to_update.isna().sum().sum() == 0

# Снимание флага 2 если не попали в расширенный список текущей недели!!!!

In [184]:
# список актуальных красных расширенный последней недели
red_webs_list_LAST_DATE = ch_query_DWH2(f"""
WITH 
    (SELECT max(report_date) AS max_added_dt 
        FROM sandbox_analytical_projects.red_webs_list) AS max_added_dt
SELECT * 
    FROM sandbox_analytical_projects.red_webs_list
    WHERE report_date = max_added_dt
""")

In [185]:
# Замена пропусков в определенных колонках
columns_to_replace = ['sub', 'sub_type']
red_webs_list_LAST_DATE[columns_to_replace] = red_webs_list_LAST_DATE[columns_to_replace].fillna("")

In [186]:
assert red_webs_list_LAST_DATE.isna().sum().sum() == 0

In [187]:
def assign_value(row):
    if row['product_name'] in ['Beta']:
        return 'ba'
    elif row['product_name'] == 'Mos':
        return 'mb'
    else:
        return 'error'

# Применение функции к DataFrame для создания новой колонки
red_webs_list_LAST_DATE['project'] = red_webs_list_LAST_DATE.apply(assign_value, axis=1)

In [188]:
assert red_webs_list_LAST_DATE[red_webs_list_LAST_DATE['project'] == 'error'].shape[0 ] == 0

In [189]:
red_webs_list_LAST_DATE = red_webs_list_LAST_DATE.rename(columns={'ID': 'pid'})

In [190]:
red_webs_list_LAST_DATE = red_webs_list_LAST_DATE[['project', 'type', 'geo', 'pid', 'sub', 'sub_type']]

In [191]:
### делим  на два датасета с флагом 2 и остальными


base_data_to_update_block_flg_2 = (base_data_to_update[
                                        base_data_to_update['block_flg'] == 2]
                                      .reset_index(drop=True)
                                             )

base_data_to_update_block_flg_ANOTHER = (base_data_to_update[
                                                base_data_to_update['block_flg'] != 2]
                                            .reset_index(drop=True)
                                                   )

assert (
    base_data_to_update_block_flg_2.shape[0] 
  + base_data_to_update_block_flg_ANOTHER.shape[0] 
 == base_data_to_update.shape[0]
               )

In [192]:
red_webs_list_LAST_DATE['block_desc'] = 2

In [193]:
# Объединение по ключевым столбцам
merged_df = base_data_to_update_block_flg_2.merge(red_webs_list_LAST_DATE, 
                                                     on=['project', 
                                                         'type', 
                                                         'geo', 
                                                         'pid', 
                                                         'sub', 
                                                         'sub_type'], 
                                                     how='left', 
                                                     suffixes=('', '_df1'))

In [194]:
merged_df['block_desc_df1'] = merged_df['block_desc_df1'].fillna(-777)
merged_df['block_desc_df1'] = merged_df['block_desc_df1'].astype(int)

In [195]:
merged_df.head()

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc,block_desc_df1
0,mb,perform,RU,347114,988,sub2,2,2025-04-23,limit=200,-777
1,mb,brand,UZ,321992,charos__gafurova,sub3,2,2025-05-07,limit=1100; речек 04.06.2025,2
2,mb,perform,RU,114170,st346,sub2,2,2025-05-21,limit=3800; речек 18.06.2025,-777
3,mb,brand,BD,340702,1913,sub2,2,2025-05-21,limit=1200; речек 18.06.2025,2
4,mb,perform,PT,295150,,,2,2025-05-21,limit=7500; речек 04.06.2025,-777


In [196]:
# Создаем словарь из merged_df (где block_flg == 2)
mask = merged_df['block_desc_df1'] == -777
pid_to_update = merged_df.loc[mask, ['project', 
                                     'type', 
                                     'geo', 
                                     'pid', 
                                     'sub', 
                                     'sub_type']].set_index(
    ['project', 'type', 'geo', 'pid', 'sub', 'sub_type']
).index

# Обновляем block_flg в concat_data_payback_2_FLG
base_data_to_update_block_flg_2.loc[
    base_data_to_update_block_flg_2.set_index(['project', 
                                                  'type', 
                                                  'geo', 
                                                  'pid', 
                                                  'sub', 
                                                  'sub_type']).index.isin(pid_to_update),
    'block_flg'
] = 0

Три ячейки ниже для проверки корректности :

In [198]:
base_data_to_update.shape[0]

3095

In [199]:
# Объединяем DataFrame по строкам
base_data_to_update = pd.concat([base_data_to_update_block_flg_2, 
                                    base_data_to_update_block_flg_ANOTHER], 
                                   ignore_index=True).reset_index(drop=True)


In [200]:
base_data_to_update.shape[0]

3095

# ПРОВЕРКА КОНФЛИКТОВ МЕЖДУ 3м ФЛАГОМ И 1м НЕ РАВНЫМ АФ:

In [202]:
### делим  на два датасета с флагом 2 и остальными


base_data_to_update_3_FLAG = (base_data_to_update[base_data_to_update['block_flg'] == 3]
                                 .reset_index(drop=True)
                                        )

base_data_to_update_1_FLAG = (base_data_to_update[base_data_to_update['block_flg'] == 1]
                                 .reset_index(drop=True)
                                        )

base_data_to_update_ALL_ANOTHER_FLAG = (
                                        base_data_to_update.loc[
                                          (base_data_to_update['block_flg'] != 1) 
                                        & (base_data_to_update['block_flg'] != 3)]
                                           .reset_index(drop=True)
                                                  )

assert (base_data_to_update_3_FLAG.shape[0] 
        + base_data_to_update_1_FLAG.shape[0] 
        + base_data_to_update_ALL_ANOTHER_FLAG.shape[0]) == base_data_to_update.shape[0]

In [203]:
base_data_to_update_ALL_ANOTHER_FLAG['block_flg'].unique()

array([0, 2, 4])

In [204]:
# Объединение по ключевым столбцам
merged_df = base_data_to_update_3_FLAG.merge(base_data_to_update_1_FLAG, 
                                                on=['project', 
                                                    'type', 
                                                    'geo', 
                                                    'pid', 
                                                    'sub', 
                                                    'sub_type'], 
                                                how='left', 
                                                suffixes=('', '_df1'))

In [205]:
FLAG_3_TO_ZERO_ANOTHER_1_FLG = merged_df.loc[(merged_df['block_flg_df1'] == 1)].reset_index(drop=True)

In [207]:
mask = merged_df['block_flg_df1'] == 1

In [208]:

mask = merged_df['block_flg_df1'] == 1
pid_to_update = merged_df.loc[mask, ['project', 
                                     'type', 
                                     'geo', 
                                     'pid', 
                                     'sub', 
                                     'sub_type']].set_index(
    ['project', 
     'type', 
     'geo', 
     'pid', 
     'sub', 
     'sub_type']
).index

# Обновляем block_flg в concat_data_payback_2_FLG
base_data_to_update_3_FLAG.loc[
    base_data_to_update_3_FLAG.set_index(['project', 
                                             'type', 
                                             'geo', 
                                             'pid', 
                                             'sub', 
                                             'sub_type']).index.isin(pid_to_update),
    'block_flg'
] = 0

In [209]:
base_data_to_update.shape[0]

3095

In [210]:
# Объединяем DataFrame по строкам
base_data_to_update = pd.concat([base_data_to_update_3_FLAG, 
                                    base_data_to_update_1_FLAG], 
                                   ignore_index=True).reset_index(drop=True)

base_data_to_update = pd.concat([base_data_to_update, 
                                    base_data_to_update_ALL_ANOTHER_FLAG], 
                                   ignore_index=True).reset_index(drop=True)

In [211]:
base_data_to_update.shape[0]

3095

# УБИРАЕМ СТРОКИ С НУЛЕВЫМИ ФЛАГАМИ ИЗ МАКСИМАЛЬНОЙ ДАТЫ В ДАТАСЕТЕ

In [212]:
base_data_to_update = base_data_to_update.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

assert base_data_to_update.isna().sum().sum() == 0

In [213]:
xxx = base_data_to_update.shape[0]

In [214]:
base_data_to_update.shape[0]

3095

In [215]:
base_data_to_update.head()

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,brand,BD,136659,6658,sub2,3,2025-04-16,limit=1; речек 16.06
1,mb,perform,HU,324248,,,3,2025-04-16,04.2025: limit=1
2,mb,brand,BD,136657,6081,sub2,3,2025-04-16,limit=1; речек 16.05
3,mb,brand,BD,136657,6573,sub2,3,2025-04-16,limit=1; речек 16.05
4,mb,brand,BD,136657,6597,sub2,3,2025-04-16,limit=1; речек 16.05


In [216]:
sss = base_data_to_update.loc[
    (base_data_to_update['blocked_date'] == base_data_to_update['blocked_date'].max()) 
                        & (base_data_to_update['block_flg'] == 0)].shape[0]

In [217]:
base_data_to_update.loc[
    (base_data_to_update['blocked_date'] == base_data_to_update['blocked_date'].max()) 
                        & (base_data_to_update['block_flg'] == 0)]

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc


In [218]:
base_data_to_update.shape[0]

3095

In [219]:
base_data_to_update = base_data_to_update.drop(
    base_data_to_update.loc[
        (base_data_to_update['blocked_date'] == base_data_to_update['blocked_date'].max()) 
        & (base_data_to_update['block_flg'] == 0)
    ].index
).reset_index(drop=True)

In [220]:
##### Для проверки что все ок удалилось!!!!!!!!!
assert xxx == (base_data_to_update.shape[0] + sss)

In [221]:
base_data_to_update.shape[0]

3095

In [222]:
##### Удаляем полные дубли на всякий случай
base_data_to_update = base_data_to_update.drop_duplicates().reset_index(drop=True)

In [223]:
base_data_to_update.shape[0]

3095

In [224]:
##### Проверка все ли ок
base_data_to_update.loc[(base_data_to_update['pid'] == 27438) 
                           & (base_data_to_update['sub'] == '')]

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
20,ba,brand,BD,27438,,,1,2025-04-03,Период мониторинга закончен. Трафик не окупаем.
2177,ba,brand,ALL,27438,,,0,2025-02-26,03.2025: limit=1
2407,ba,brand,BD,27438,,,0,2025-03-26,limit=100


In [225]:
##### Проверка все ли ок
base_data_to_update.loc[(base_data_to_update['pid'] == 336242) 
                           & (base_data_to_update['sub'] == '')]

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
1477,mb,perform,RU,336242,,,1,2025-04-03,Период мониторинга закончен. Трафик не окупаем.
1815,mb,perform,TR,336242,,,1,2025-04-16,Медиабаинг: Трафик остановлен
2358,mb,perform,RU,336242,,,0,2025-03-13,limit=100
2388,mb,perform,TR,336242,,,0,2025-03-19,limit=100


# Временный костыль по просьбе заказчика, т.к. в табличке АФ есть дубли!

In [226]:
base_data_to_update = (base_data_to_update.drop_duplicates(subset=['project',
                                                                        'type', 
                                                                        'geo', 
                                                                        'pid', 
                                                                        'sub', 
                                                                        'sub_type', 
                                                                        'block_flg'])
                                                .reset_index(drop = True)
                                 )

# Пункт №1: Предварительная проверка данных (не должно быть что бы лимит стоял и он стопнут):

In [228]:
_block_flg_2 = base_data_to_update[base_data_to_update['block_flg'] == 2].reset_index(drop=True)
ALL_ANOTHER_block_flg = (base_data_to_update[base_data_to_update['block_flg'] != 2]
                             .reset_index(drop=True)
                                )

## Блок проверки на дубли записей по лимитам:

Ниже должны быть пустые датасеты!!!!!!

In [229]:
### Дубликаты если есть получившиеся
assert _block_flg_2[_block_flg_2.duplicated(['project', 
                                             'type', 
                                             'geo', 
                                             'pid', 
                                             'sub', 
                                             'sub_type', 
                                             'block_flg'])].shape[0] == 0
_block_flg_2[_block_flg_2.duplicated(['project', 
                                      'type', 
                                      'geo', 
                                      'pid', 
                                      'sub', 
                                      'sub_type', 
                                      'block_flg'])].sort_values(by = 'geo')

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc


In [230]:
_block_flg_3 = base_data_to_update[base_data_to_update['block_flg'] == 3].reset_index(drop=True)
### Дубликаты если есть получившиеся
assert _block_flg_3[_block_flg_3.duplicated(['project', 
                                             'type', 
                                             'geo', 
                                             'pid', 
                                             'sub', 
                                             'sub_type', 
                                             'block_flg'])].shape[0] == 0
_block_flg_3[_block_flg_3.duplicated(['project', 
                                      'type', 
                                      'geo', 
                                      'pid', 
                                      'sub', 
                                      'sub_type', 
                                      'block_flg'])].sort_values(by = 'geo')

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc


In [231]:
_block_flg_4 = base_data_to_update[base_data_to_update['block_flg'] == 4].reset_index(drop=True)
### Дубликаты если есть получившиеся
assert _block_flg_4[_block_flg_4.duplicated(['project', 
                                             'type', 
                                             'geo', 
                                             'pid', 
                                             'sub', 
                                             'sub_type', 
                                             'block_flg'])].shape[0] == 0
_block_flg_4[_block_flg_4.duplicated(['project', 
                                      'type', 
                                      'geo', 
                                      'pid', 
                                      'sub', 
                                      'sub_type', 
                                      'block_flg'])].sort_values(by = 'geo')

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc


In [232]:
_block_flg_1 = base_data_to_update[base_data_to_update['block_flg'] == 1].reset_index(drop=True)
### Дубликаты если есть получившиеся
assert _block_flg_1[_block_flg_1.duplicated(['project', 
                                             'type', 
                                             'geo', 
                                             'pid', 
                                             'sub', 
                                             'sub_type', 
                                             'block_flg'])].shape[0] == 0
_block_flg_1[_block_flg_1.duplicated(['project', 
                                      'type', 
                                      'geo', 
                                      'pid', 
                                      'sub', 
                                      'sub_type', 
                                      'block_flg'])].sort_values(by = 'geo')

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc


In [233]:
# Находим пересечения
common_rows = ALL_ANOTHER_block_flg[ALL_ANOTHER_block_flg['block_flg'] == 1].merge(_block_flg_2, 
                                                                                   on=['project', 
                                                                                       'type', 
                                                                                       'geo', 
                                                                                       'pid', 
                                                                                       'sub', 
                                                                                       'sub_type'], 
                                                                                   how='inner')
assert common_rows.shape[0] == 0

# Находим пересечения
common_rows = ALL_ANOTHER_block_flg[ALL_ANOTHER_block_flg['block_flg'] == 1].merge(_block_flg_3, 
                                                                                   on=['project', 
                                                                                       'type', 
                                                                                       'geo', 
                                                                                       'pid', 
                                                                                       'sub', 
                                                                                       'sub_type'], 
                                                                                   how='inner')
assert common_rows.shape[0] == 0

# Находим пересечения
common_rows = ALL_ANOTHER_block_flg[ALL_ANOTHER_block_flg['block_flg'] == 1].merge(_block_flg_4, 
                                                                                   on=['project', 
                                                                                       'type', 
                                                                                       'geo', 
                                                                                       'pid', 
                                                                                       'sub', 
                                                                                       'sub_type'], 
                                                                                   how='inner')
assert common_rows.shape[0] == 0

###############################################################################

###### Проверка на дубли по остальным не нулевым флагам!!!!!!!!!!

# Находим пересечения
common_rows = base_data_to_update[base_data_to_update['block_flg'] == 2].merge(_block_flg_3, 
                                                                                     on=['project', 
                                                                                         'type', 
                                                                                         'geo', 
                                                                                         'pid', 
                                                                                         'sub', 
                                                                                         'sub_type'], 
                                                                                     how='inner')
assert common_rows.shape[0] == 0

# Находим пересечения
common_rows = ALL_ANOTHER_block_flg[ALL_ANOTHER_block_flg['block_flg'] == 3].merge(_block_flg_4, 
                                                                                   on=['project', 
                                                                                       'type', 
                                                                                       'geo', 
                                                                                       'pid', 
                                                                                       'sub', 
                                                                                       'sub_type'], 
                                                                                   how='inner')
assert common_rows.shape[0] == 0

# Находим пересечения
common_rows = ALL_ANOTHER_block_flg[ALL_ANOTHER_block_flg['block_flg'] == 4].merge(_block_flg_2, 
                                                                                   on=['project', 
                                                                                       'type', 
                                                                                       'geo', 
                                                                                       'pid', 
                                                                                       'sub', 
                                                                                       'sub_type'], 
                                                                                   how='inner')
assert common_rows.shape[0] == 0

# Находим пересечения
common_rows = ALL_ANOTHER_block_flg[ALL_ANOTHER_block_flg['block_flg'] == 4].merge(_block_flg_3, 
                                                                                   on=['project', 
                                                                                       'type', 
                                                                                       'geo', 
                                                                                       'pid', 
                                                                                       'sub', 
                                                                                       'sub_type'], 
                                                                                   how='inner')
assert common_rows.shape[0] == 0

# Находим пересечения
common_rows = base_data_to_update[base_data_to_update['block_flg'] == 2].merge(_block_flg_4, 
                                                                                     on=['project', 
                                                                                         'type', 
                                                                                         'geo', 
                                                                                         'pid', 
                                                                                         'sub', 
                                                                                         'sub_type'], 
                                                                                     how='inner')
assert common_rows.shape[0] == 0

In [234]:
####### Проверка на дубли в нутри одного флага, не равного нулевому

sss = base_data_to_update[base_data_to_update['block_flg'] == 1]
### Дубликаты если есть получившиеся
assert sss[sss.duplicated(['project', 
                           'type', 
                           'geo', 
                           'pid', 
                           'sub', 
                           'sub_type', 
                           'block_flg'])].shape[0] == 0

sss = base_data_to_update[base_data_to_update['block_flg'] == 2]
### Дубликаты если есть получившиеся
assert sss[sss.duplicated(['project', 
                           'type', 
                           'geo', 
                           'pid', 
                           'sub', 
                           'sub_type', 
                           'block_flg'])].shape[0] == 0

sss = base_data_to_update[base_data_to_update['block_flg'] == 3]
### Дубликаты если есть получившиеся
assert sss[sss.duplicated(['project', 
                           'type', 
                           'geo', 
                           'pid', 
                           'sub', 
                           'sub_type', 
                           'block_flg'])].shape[0] == 0

sss = base_data_to_update[base_data_to_update['block_flg'] == 4]
### Дубликаты если есть получившиеся
assert sss[sss.duplicated(['project', 
                           'type', 
                           'geo', 
                           'pid', 
                           'sub', 
                           'sub_type', 
                           'block_flg'])].shape[0] == 0

# Пункт общей проверки завершен, данные в таблице корректны.

In [235]:
base_data_to_update

,project,type,geo,pid,sub,sub_type,block_flg,blocked_date,block_desc
0,mb,brand,BD,136659,6658,sub2,3,2025-04-16,limit=1; речек 16.06
1,mb,perform,HU,324248,,,3,2025-04-16,04.2025: limit=1
2,mb,brand,BD,136657,6081,sub2,3,2025-04-16,limit=1; речек 16.05
3,mb,brand,BD,136657,6573,sub2,3,2025-04-16,limit=1; речек 16.05
4,mb,brand,BD,136657,6597,sub2,3,2025-04-16,limit=1; речек 16.05
...,...,...,...,...,...,...,...,...,...
3059,mb,perform,TR,49706,ig175,sub1,4,1980-01-01,Скам!
3060,mb,brand,IN,49534,23044,sub2,4,1980-01-01,Скам!
3061,mb,perform,TR,49710,bonuslion,sub1,4,1980-01-01,Скам!
3062,mb,perform,AZ,49664,ali.nadiroglu,sub2,4,1980-01-01,Скам!


In [236]:
base_data_to_update = base_data_to_update.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

assert base_data_to_update.isna().sum().sum() == 0

# Зануляем старые единички по прошествии 3х месяцев:

In [237]:
# Текущая дата минус 3 месяца
three_months_ago = datetime.today() - relativedelta(months=3)

# Форматируем в строку 'YYYY-MM-DD'
three_months_ago_str = three_months_ago.strftime('%Y-%m-%d')

#print(three_months_ago_str)

In [238]:
three_months_ago_str

'2025-03-19'

In [239]:
base_data_to_update.loc[(base_data_to_update['block_flg'] == 1) 
                            & (base_data_to_update['blocked_date'] < three_months_ago_str)
                            & (base_data_to_update['block_desc'] !=  'Антифрод: Заблокировано'), 
                           'block_flg'] = 0

assert (base_data_to_update.loc[(base_data_to_update['block_flg'] == 1) 
                            & (base_data_to_update['blocked_date'] < three_months_ago_str)
                            & (base_data_to_update['block_desc'] !=  'Антифрод: Заблокировано')]
        .shape[0] == 0)

assert (base_data_to_update.loc[(base_data_to_update['block_flg'] == 1) 
                            & (base_data_to_update['blocked_date'] > three_months_ago_str)
                            & (base_data_to_update['block_desc'] !=  'Антифрод: Заблокировано')]
        .shape[0] != 0)

# Перестраховка перед загрузкой в валинор!!

In [240]:
base_data_to_update.to_excel('base_data_to_update_ПЕРЕСТРАХОВКА.xlsx', index=False)

# Пункт если был сбой в БД и загрузка по каким-то причинам не завершилась

Данные ячейки активировать только в экстренных случаях или по появлению соответсатвующих AD-Hoc задач!

In [ ]:
base_data_to_update = pd.read_excel('base_data_to_update_ПЕРЕСТРАХОВКА.xlsx', dtype={'sub': 'str'})

In [ ]:
# Замена пропусков в определенных колонках
columns_to_replace = ['sub', 'sub_type']
base_data_to_update[columns_to_replace] = base_data_to_update[columns_to_replace].fillna("")

In [ ]:
base_data_to_update = base_data_to_update.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

assert base_data_to_update.isna().sum().sum() == 0

In [ ]:
base_data_to_update.shape[0]

# ЗАГРУЗКА ОБНОВЛЕННОЙ ТАБЛИЦЫ В БД

In [20]:
base_data_to_update = base_data_to_update.astype({
    'project': 'str',
    'type': 'str',
    'geo': 'str',
    'pid': 'int64',
    'sub': 'str',
    'sub_type': 'str',
    'block_flg': 'int64',
    'blocked_date': 'datetime64[ns]',
    'block_desc': 'str'
})

assert base_data_to_update.isna().sum().sum() == 0

In [21]:
### Настраиваем соединение с БД
creds = pd.read_excel('Data_base.xlsx', index_col=0)


In [22]:
table_name = 'black_list_payback'

In [23]:
# учетные данные для загрузки в валинор
config_base = {
    'host': 'base.dats.tech',
    'user': creds.loc['base', 'user'],
    'password': creds.loc['base', 'password'],
    'database': 'reports_sharded'}

# функция для очистки таблицы и загрузки данных в схему reports
def base_upload(table_name, df, clickhouse_config=config_base):
    with dbapi.connect(**clickhouse_config) as clickhouse_conn:
        df = df.fillna(0).round(2)
        # разбиваем дф для загрузки на лист тьюплов
        rows = list(df.itertuples(index=False, name=None))
        # создаем курсор
        cursor = clickhouse_conn.cursor()
        # транкейтим таблицу в схемах
        cursor.execute(f'TRUNCATE TABLE reports_sharded."{table_name}" on cluster mostbet_bi')
        cursor.execute(f'TRUNCATE TABLE reports."{table_name}" on cluster mostbet_bi')
        # загружаем данные в схему reports на валинор
        query = f'INSERT INTO reports."{table_name}" VALUES'
        try:
            cursor.execute(
                query,
                rows
            )
            clickhouse_conn.commit()
            print(f'{table_name} done')
        except Exception as exe:
            # перестраховка (пара таблиц не записываются способом выше)
            try:
                print(f'{table_name} second attempt {exe}')
                x = 0
                for i in range(np.ceil(len(rows)/5000)):
                    query = f'INSERT INTO reports."{table_name}" VALUES'
                    for j in range(x,x+5000):
                        try:
                            query += f"\n{rows[j]},"
                        except Exception as e:
                            break
                    cursor.execute(query)
                    x += 5000
                print(f'{table_name} done')
            except Exception as er:
                print(f'{table_name} failed, {er}')



In [24]:
base_upload(table_name, base_data_to_update)

black_list_payback done


# Добавляем отбивку в ТГ бот:

In [247]:

FLAG_3_TO_ZERO_ANOTHER_1_FLG = FLAG_3_TO_ZERO_ANOTHER_1_FLG.rename(columns={'block_flg': 'OLD_block_flg',
                                                     'blocked_date': 'OLD_blocked_date',
                                                     'block_flg_df1': 'NEW_block_flg',
                                                     'blocked_date_df1': 'NEW_blocked_date',
                                                     'block_desc_df1': 'block_desc:'})

FLAG_2_3_BY_ZERO = FLAG_2_3_BY_ZERO.rename(columns={'block_flg': 'OLD_block_flg',
                                                     'blocked_date': 'OLD_blocked_date',
                                                     'block_flg_df1': 'NEW_block_flg',
                                                     'blocked_date_df1': 'NEW_blocked_date',
                                                     'block_desc_df1': 'block_desc:'})


In [248]:
# Зануленные 3 флаги от остальных флагов равных 1 скриптом
FLAG_3_TO_ZERO_ANOTHER_1_FLG[['block_desc:', 
                              'project', 
                              'type', 
                              'geo', 
                              'pid', 
                              'sub', 
                              'sub_type', 
                              'OLD_block_flg', 
                              'NEW_block_flg', 
                              'NEW_blocked_date']]

,block_desc:,project,type,geo,pid,sub,sub_type,OLD_block_flg,NEW_block_flg,NEW_blocked_date


In [249]:
#### Кейсы где дужно занулить 2й и 3й флаг, т.к. есть первый!!!!
FLAG_2_3_BY_ZERO[['block_desc:', 
                  'project', 
                  'type', 
                  'geo', 
                  'pid', 
                  'sub', 
                  'sub_type', 
                  'OLD_block_flg', 
                  'NEW_block_flg', 
                  'NEW_blocked_date']]

,block_desc:,project,type,geo,pid,sub,sub_type,OLD_block_flg,NEW_block_flg,NEW_blocked_date


In [250]:
FLAG_2_3_BY_ZERO['block_desc:'].unique()

array([], dtype=object)

In [251]:
message = f"🔊<b>black_list_payback:</b>\n\n👨‍💻\nЗануленные 3 флаги от первых флагов: {FLAG_3_TO_ZERO_ANOTHER_1_FLG.shape[0]};\nПрочие кейсы где был занулен 2й и 3й флаг, т.к. есть первый: {FLAG_2_3_BY_ZERO.shape[0]}\nКоличественное значение строк скама: {pids_list_data.shape[0]}\n\n<b><u>Данные по всем флагам обновлены!</u></b> ✅"

In [252]:
def bot_channel_alert(message):
    channel_id = -1009284606397 # TG channel
    bot_token = "7002340744:AAGSceUHIsnoV0JfJCsQ14" 
    bot = telebot.TeleBot(bot_token)
    bot.send_message(channel_id, message, parse_mode="HTML")

In [253]:
bot_channel_alert(message)

# THE END